In [1]:
from cng.utils import *
import ibis
from ibis import _
import pathlib

con = ibis.duckdb.connect(extensions = ["spatial", "h3"])
set_secrets(con)
duckdb_install_h3()


In [ ]:
def geom_to_h3(con, 
               taxa, 
               cols = "taxon_id, parent_taxon_id, name, rank, iconic_taxon_id, iconic_taxon_name",
               zoom = "4"):

    con.read_geo(gpkg, taxa) # FIXME allow overwrite

    
    con.sql(f'''
      WITH t2 AS (
        WITH t1 AS (
          SELECT {cols},  ST_Dump(geom) AS geom 
          FROM {taxa}
        ) 
        SELECT {cols},
              h3_polygon_wkt_to_cells_string(UNNEST(geom).geom, {zoom}) AS h{zoom}
        FROM t1
      )
      SELECT {cols}, UNNEST(h{zoom}) AS h{zoom} FROM t2
      ''').to_parquet(f"s3://public-inat/hex/{taxa}.parquet")




In [ ]:
## assumes files have all been downladed from https://www.inaturalist.org/pages/range_maps
files = pathlib.Path('~/nvme/public-inat').expanduser().rglob('*.gpkg')

for f in files:
    taxa = f.name.removesuffix('.gpkg').replace('iNaturalist_geomodel_', '')
    gpkg = f"/home/jovyan/nvme/public-inat/iNaturalist_geomodel_{taxa}.gpkg"
    taxon = taxa
    con.read_geo(gpkg, taxa).to_parquet(f"s3://public-inat/polygon/{taxa}.parquet")


In [ ]:

files = pathlib.Path('~/nvme/public-inat').expanduser().rglob('*.gpkg')

for f in files:
    taxa = f.name.removesuffix('.gpkg').replace('iNaturalist_geomodel_', '')
    gpkg = f"/home/jovyan/nvme/public-inat/iNaturalist_geomodel_{taxa}.gpkg"
    taxon = taxa
    geom_to_h3(con, taxa)

In [2]:
# access all range-maps, h3
con.read_parquet("s3://public-inat/hex/**").head().execute()

,taxon_id,parent_taxon_id,name,rank,iconic_taxon_id,iconic_taxon_name,h4
0,47174,47175,Nelusetta ayraud,species,47178,Actinopterygii,84a700dffffffff
1,47174,47175,Nelusetta ayraud,species,47178,Actinopterygii,84a7047ffffffff
2,47174,47175,Nelusetta ayraud,species,47178,Actinopterygii,84a7041ffffffff
3,47174,47175,Nelusetta ayraud,species,47178,Actinopterygii,84a7007ffffffff
4,47174,47175,Nelusetta ayraud,species,47178,Actinopterygii,84c9931ffffffff


In [ ]:
# Taxonomy

